In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from dask.distributed import Client
import pprint 
%run /home/jovyan/escience-2022/Tjaernoe2022-group2/notebooks/functions.ipynb
%matplotlib inline

In [6]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:38381")
client

<Client: 'tcp://127.0.0.1:38381' processes=8 threads=32, memory=125.61 GiB>

## Create weights

In [3]:
def computeWeightedMean(ds):
    # Compute weights based on the xarray you pass
    weights = np.cos(np.deg2rad(ds.lat))
    weights.name = "weights"
    # Compute weighted mean
    air_weighted = ds.weighted(weights)
    weighted_mean = air_weighted.mean(("lon", "lat"))
    return weighted_mean

In [7]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
waccm = col.search(source_id=['CESM2-WACCM'], experiment_id=['historical'], table_id=['Amon'], variable_id=['o3'])
cam = col.search(source_id=['CESM2'], experiment_id=['historical'], table_id=['Amon'], variable_id=['o3'])

waccm.df
cam.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2-WACCM,historical,r1i1p1f1,Amon,o3,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2-WACCM/histori...,NaN,20190415
1,CMIP,NCAR,CESM2-WACCM,historical,r2i1p1f1,Amon,o3,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2-WACCM/histori...,NaN,20190415
2,CMIP,NCAR,CESM2-WACCM,historical,r3i1p1f1,Amon,o3,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2-WACCM/histori...,NaN,20190415


In [2]:
ds_dict_waccm = waccm.to_dataset_dict(zarr_kwargs={'use_cftime':True}, cdf_kwargs={"chunks": {"time": 4}})
ds_dict_cam = cam.to_dataset_dict(zarr_kwargs={'use_cftime':True},cdf_kwargs={"chunks": {"time": 4}})

NameError: name 'waccm' is not defined

In [13]:
print(list(ds_dict_waccm.keys()))
print(list(ds_dict_cam.keys()))
ds_waccm = ds_dict_waccm['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
ds_cam = ds_dict_cam['CMIP.NCAR.CESM2.historical.Amon.gn']

['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
['CMIP.NCAR.CESM2.historical.Amon.gn']


## Select time period

In [15]:
start = cftime.DatetimeNoLeap(1985,1,15)
end = cftime.DatetimeNoLeap(1996,1,15)

# WACCM
ds_waccm_1986_1996 = ds_waccm.sel(time=slice(start, end))
# CAM
ds_cam_1986_1996 = ds_cam.sel(time=slice(start, end))

## Calculate ozone from concentration to Dobson units

In [ ]:
# Creates a new variable called totO3 for WACCM and CAM 
ds_waccm_1986_1996 = calculate_total_ozone_p(ds_waccm_1986_1996)
#ds_cam_1986_1996 = calculate_total_ozone_p(ds_cam_1986_1996)

Minimum column ozone value: 58.61533162268342
